In [ ]:
!export XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda
!export CUDA_DIR="/usr/lib/cuda"
!export TF_GPU_ALLOCATOR=cuda_malloc_async

In [ ]:
import tensorflow as tf
tf.config.threading.set_inter_op_parallelism_threads(8)
tf.config.threading.set_intra_op_parallelism_threads(8)
physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_virtual_device_configuration(physical_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8000)])
# tf.config.experimental.set_virtual_device_configuration(physical_devices[1], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)
# print("GPUS: {}".format(len(physical_devices)))

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow import keras
import time
import tensorflow_addons as tfa

layers = tf.keras.layers

from IPython import display

In [ ]:
# (train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
# train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
# train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]
# !rm -rf ./logs

In [ ]:
def normalize(image):
    return (image -127.5) / 127.5

def denormalize(image):
    return tf.cast(image * 127.5 + 127.5, np.uint8).numpy()

In [ ]:
import pathlib
import tensorflow as tf
import random

data_dir = pathlib.Path("./prepa")
pictures = list(data_dir.glob('*.png'))
pictures.sort()
pictures


data_dir = pathlib.Path("./people")
large_pictures = list(data_dir.glob('*.jpg'))
large_pictures.sort()
large_pictures[:5]

In [ ]:
input_w = 720
input_h = 480


# input_h = 672
# input_w = 976

# input_h *= 1.2
# input_w *= 1.2


# input_h = 576
# input_w = 864

hwfactor = input_h / input_w

input_h, input_w

BUFFER_SIZE = 800
BATCH_SIZE = 10


BUFFER_SIZE = 10
BATCH_SIZE = 3

In [ ]:
input_h = int(input_h)
input_w = int(input_w)
input_h, input_w

# Enlarge image

In [ ]:
rf = 1
resize_factor = 1/rf # about 0.0833
resized_size_h = int(input_h * resize_factor)
resized_size_w = int(input_w * resize_factor)

print(resized_size_h, resized_size_h * rf, input_h)
assert resized_size_h * rf == input_h

print(resized_size_w, resized_size_w * rf, input_w)
assert resized_size_w * rf == input_w

In [ ]:
image_count = len(pictures)
large_image_count = len(large_pictures) + len(large_pictures) * 20 * 2
print(f"image_count={image_count}, large_image_count={large_image_count}")

image_count += large_image_count

def make_dataset(pictures, large_pictures):
  def load_large_images():
    for filename in large_pictures:
      raw_png = tf.io.read_file(str(filename), name=filename)
      decoded_png = tf.image.decode_jpeg(raw_png, channels=3, name=filename)
      cropped = tf.image.resize_with_crop_or_pad(
          decoded_png,input_h ,input_w
      )
      yield cropped

      for i in range(20):
        cropped = tf.image.random_crop(
          decoded_png, size=[input_h, input_w, 3])
        yield cropped
        # yield tf.image.random_flip_left_right(cropped)

    for filename in pictures:
        raw_png = tf.io.read_file(str(filename), name=filename)
        decoded_png_2 = tf.image.decode_png(raw_png, channels=3, name=filename)
        decoded_png_2 = tf.image.resize(decoded_png_2, [input_h, input_w],
                          method=tf.image.ResizeMethod.BILINEAR)
        
        yield decoded_png_2
        yield tf.image.flip_left_right(decoded_png_2)

        

  train_ds = tf.data.Dataset.from_generator(load_large_images,  output_signature=
         tf.TensorSpec(shape=(input_h, input_w, 3), dtype=tf.float16)).map(normalize)

  def resize_and_couple(images):
    # return (images,images)
    down = tf.image.resize(
        images,
        [int(resized_size_h / 8), int(resized_size_w / 8)],
        preserve_aspect_ratio=True,
        antialias=False,
        name=None)


    return (images, tf.image.resize(
        down,
        [resized_size_h, resized_size_w],
        preserve_aspect_ratio=True,
        antialias=False,
        name=None))

  zipped_train_dataset = train_ds.interleave(
      lambda x: tf.data.Dataset.from_tensors(x).map(resize_and_couple, num_parallel_calls=tf.data.AUTOTUNE),
      cycle_length=4, num_parallel_calls=tf.data.AUTOTUNE,
      deterministic=False
  )
  # Batch and shuffle the data
  return zipped_train_dataset
  # train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(1)

# lp_imgs = list(train_ds.shuffle(200).take(2))
train_ds = make_dataset(pictures, large_pictures).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# i = 1
# for img_ind in range(len(lp_imgs)):
#     plt.figure(figsize=(12, 9))
#     img = denormalize(lp_imgs[img_ind])
#     plt.imshow(img)
#     plt.axis('off')
#     i += 1


In [ ]:
def get_random_samples(size=2):
    some_pictures = random.choices(pictures, k=size)
    some_large_pictures = random.choices(large_pictures, k=size) 
    return make_dataset(some_pictures, some_large_pictures)

In [ ]:
resized_size_h

In [ ]:
model_input_w = input_w
model_input_h = input_h

model_input_w, model_input_h

In [ ]:
resized_size_w, resized_size_h

In [ ]:
input_w, input_h

In [ ]:
def closest(stride, base_kernel, K):
    lst = list(range(0,base_kernel+stride, stride))
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

closest(2, 6, 5)

In [ ]:
upsamples_per_scale = {
    2: 1,
    4: 2,
    8: 3
}


def pixel_shuffle(scale):
    return lambda x: tf.nn.depth_to_space(x, scale)


def upsample(x_in, num_filters):
    x = layers.Conv2D(num_filters, kernel_size=3, padding='same')(x_in)
    x = layers.Lambda(pixel_shuffle(scale=2))(x)
    return layers.PReLU(shared_axes=[1, 2])(x)

init_fn = tf.keras.initializers.LecunNormal(seed=123)


def residual_block(block_input, num_filters, momentum=0.8):
    x = layers.Conv2D(num_filters, kernel_size=3, padding='same', kernel_initializer=init_fn)(block_input)
    x = layers.BatchNormalization(momentum=momentum)(x)
    x = layers.PReLU(shared_axes=[1, 2])(x)
    x = layers.Conv2D(num_filters, kernel_size=3, padding='same', kernel_initializer=init_fn)(x)
    x = layers.BatchNormalization(momentum=momentum)(x)
    return x

def make_sgenerator_model(scale=8, num_filters=64):
    # needs to be divisible by the stride to avoid checkerboard patterns
    base_f = 6
    def make_kernel(s):
        return closest(min(2,s), s, s*hwfactor)
    kernel_size = make_kernel(base_f)
    num_upsamples = upsamples_per_scale[scale]

    lr = tf.keras.Input(shape=(None, None, 3))
    x = layers.Conv2D(32, kernel_size=make_kernel(3), padding='same', kernel_initializer=init_fn, activation='leaky_relu')(lr)
    x = layers.BatchNormalization()(x)
    x = tf.nn.leaky_relu(x)
    x = layers.Concatenate()([x,lr])

    f = x

    x = layers.Conv2D(32, kernel_size=make_kernel(3), padding='same', kernel_initializer=init_fn, activation='leaky_relu')(lr)
    x = layers.BatchNormalization()(x)
    x = tf.nn.leaky_relu(x)
    x = layers.Concatenate()([x,f])
    
    f = x

    x = layers.Conv2D(32, kernel_size=make_kernel(3), padding='same', kernel_initializer=init_fn, activation='leaky_relu')(lr)
    x = layers.BatchNormalization()(x)
    x = tf.nn.leaky_relu(x)
    x = layers.Concatenate()([x,f])

    x = layers.Conv2D(3, kernel_size=9, padding='same', kernel_initializer=init_fn, activation='tanh')(x)
    return tf.keras.Model(lr, x, name="generator")
make_sgenerator_model()

In [ ]:
import math

def get_grid_size(x):
    col = int(math.sqrt(x))
    row = int(x / col)

    y = int(x - (col*row))
    row+=y

    return row, col

def get_bi_column(x, col=2):
    row = int(x / col)

    y = int(x - (col*row))
    row+=y

    return row, col

get_bi_column(9)

In [ ]:
import random

random_samples_ds = get_random_samples(10)

In [ ]:
initializer = tf.keras.initializers.LecunNormal()
initializer(shape=(2, 2))

In [ ]:
def conv(filters=3, kernel_size=(1,1), name=None, kernel_initializer="Zeros", normalize=False, activation=None, strides=(1,1)):
    act = {
        "leaky_relu": tf.nn.leaky_relu,
        "tanh": tf.nn.tanh
        }
    def fn(x):
        x = layers.Conv2D(3, kernel_size=kernel_size, kernel_initializer=kernel_initializer, strides=strides, padding="same", name=name)(x)
        if (normalize):
            x = layers.BatchNormalization()(x)
        if activation and activation in act:
            print(activation)
            x = act[activation](x)
        return x
    return fn

In [ ]:
def test_cnvs():
    base_f = 1
    kernel_size = closest(base_f, base_f, base_f*hwfactor)

    kernel_size_100 = closest(100, 100, 100*hwfactor)

    convs = [
        # ("Zeros (kernel=100)", conv(kernel_size=kernel_size_100)),
        # ("Zeros", conv(kernel_size=kernel_size)),
        # ("Zeros (norm)", conv(kernel_size=kernel_size, normalize=True)),
        # ("Zeros (norm & relu)", conv(kernel_size=kernel_size, normalize=True, activation="leaky_relu")),
        # ("Zeros (norm & relu, stride=10)", conv(kernel_size=kernel_size, normalize=True, strides=(10,10), activation="leaky_relu")),
        ("LecunNormal", conv(kernel_initializer=tf.keras.initializers.LecunNormal(seed=123))),
        ("LecunNormal (kernel=3)", conv(kernel_size=closest(3, 3, 3*hwfactor), kernel_initializer=tf.keras.initializers.LecunNormal(seed=123))),
        ("LecunNormal (norm)", conv(kernel_initializer=tf.keras.initializers.LecunNormal(seed=123), normalize=True)),
        ("LecunNormal (norm & relu)", conv(kernel_initializer=tf.keras.initializers.LecunNormal(seed=123), normalize=True, activation="leaky_relu")),
        ("LecunNormal (kernel=3, norm & relu)", conv(kernel_size=closest(3, 3, 3*hwfactor), kernel_initializer=tf.keras.initializers.LecunNormal(seed=123), normalize=True, activation="leaky_relu")),
        # ("VarianceScaling", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.VarianceScaling())),
        # ("VarianceScaling (norm)", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.VarianceScaling(), normalize=True)),
        # ("VarianceScaling (norm & relu)", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.VarianceScaling(), normalize=True, activation="leaky_relu")),
        # ("LecunUniform", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.LecunUniform())),
        # ("LecunUniform (norm)", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.LecunUniform(), normalize=True)),
        # ("LecunUniform (norm & relu)", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.LecunUniform(), normalize=True, activation="leaky_relu"))
    ]


    for images, resized_images in random_samples_ds.shuffle(125).take(2):
        plt.figure()
        # plt.figure(figsize=(12,4))
       
        # im[:,:,1] = 1
        # im[:,:,2] = 1
        # plt.imshow(im)
        # print(images.shape)
        # down_model = downsample(3, 4)
        # down_result = down_model(tf.cast(tf.expand_dims(images, axis=0), np.float32))
        # print (down_result.shape)

        # plt.figure()
        # plt.title("down_result")
        # plt.imshow(denormalize(np.squeeze(down_result)))

        # up_model = upsample(3, 4)
        # up_result = up_model(down_result)
        # print (up_result.shape)

        # plt.figure()
        # plt.title("up_result")
        # plt.imshow(denormalize(np.squeeze(up_result)))
        
        
        for (cname, cfn) in convs:
            im = np.copy(images)
            conv_image = tf.squeeze(cfn(tf.cast(tf.expand_dims(im, axis=0), np.float16)))
            pp = lambda x: (np.max(np.unique(x)), np.min(np.unique(x)))
            print(cname, pp(im), pp(conv_image))
            plt.figure(figsize=(12,4))
            plt.subplot(1,3,1)
            plt.title("Original")
            plt.imshow(denormalize(im))
            plt.subplot(1,3,2)
            plt.title(f"Conv: {cname}")
            plt.imshow(denormalize(conv_image))
            plt.axis('off')
            plt.subplot(1,3,3)
            plt.title(f"Original + Conv: {cname}")
            if conv_image.shape != im.shape:
                im = tf.image.resize(im, [conv_image.shape[0], conv_image.shape[1]])
            plt.imshow(denormalize(tf.add(conv_image, im)))
            plt.axis('off')

# test_cnvs()

In [ ]:


def test_generator(generator, discriminator, size=2):
    # test_dataset = tf.data.Dataset.zip((train_ds, resized_ds))
    

    sample = random.randrange(0, image_count-size-1)
    test_data = random_samples_ds.shuffle(sample).take(size).batch(size)
    for images, resized_images in test_data:
        generated_images = generator(resized_images, training=False)

        print(f"images={images.shape}, resized_images={resized_images.shape}, generated_images={generated_images.shape}")

        imgs = zip(images, resized_images, generated_images)

        col, row = get_bi_column(size*3, 3)

        # print(f"col={col}, row={row}")

        fig = plt.figure(figsize=(row * 10, col * 7))

        i = 0
        for img_set in imgs:
            for img in img_set:
                plt.subplot(col, row, i+1)
                im = np.copy(img)
                im = denormalize(im)
                plt.imshow(im)
                plt.axis('off')
                i += 1

        decision = discriminator(generated_images)
        print (f"Decision for the scaled images: {decision}")


    return generated_images

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[input_h, input_w, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model()
generator = make_sgenerator_model()

generated_image = test_generator(generator, discriminator, 2)



In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [ ]:
learning_rate=tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=[1000, 10000], values=[1e-2, 1e-4, 1e-5])
generator_optimizer = tf.keras.optimizers.Adam(learning_rate)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate)


In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])


# Metrics

In [ ]:
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = f'logs/gan/train/{current_time}'
train_log_dir

In [ ]:
checkpoint_dir = './gan4_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=5)
try:
    if ckpt_manager.latest_checkpoint:
        checkpoint.restore(ckpt_manager.latest_checkpoint)
except:
    print("Could not restore the checkopint")

In [ ]:
def generate_and_save_images(generator, discriminator, epoch, save=True): 
  predictions = test_generator(generator, discriminator, 3)
  if save == True:
    plt.savefig('./gan_output/image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()


generate_and_save_images(generator, discriminator, 0, False)

In [ ]:
import signal
import sys

def sigint_handler(signal, frame):
    print ('KeyboardInterrupt is caught')
    checkpoint.save(file_prefix = checkpoint_prefix)
    sys.exit(0)
signal.signal(signal.SIGINT, sigint_handler)

In [ ]:
# gpus = tf.config.list_logical_devices('GPU')

# with tf.device(gpus[0].name):

class GANModel(tf.keras.Model):
  def __init__(self, gen, disc):
    super(GANModel, self).__init__(name="GANModel")
    self.generator = gen
    self.discriminator = disc

  def train_step(self, all_images):
    images, resized = all_images
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(resized, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

        _gen_loss = gen_loss
        
        images = tf.cast(images, tf.float32)
        
        ssim = tf.math.reduce_sum(tf.image.ssim(images, generated_images, 255.0))
        ms_ssim = tf.math.reduce_sum(tf.image.ssim_multiscale(images, generated_images, 255.0))
        
        gen_loss += (1.0 - ssim) + (1.0 - ms_ssim)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return {"gen_loss": _gen_loss, "disc_loss":disc_loss, "ssim":ssim, "ms_ssim":ms_ssim}

In [ ]:
# %tensorboard --logdir logs/gradient_tape

In [ ]:
def on_epoch_end(epoch, logs=None):
    generate_and_save_images(generator, discriminator,
                    epoch + 1)
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                                ckpt_save_path))
# def on_batch_end(batch, logs=None):
#     if batch % 10 == 0:
#         train_log_dir = 'logs/gan/train'
#         train_summary_writer = tf.summary.create_file_writer(train_log_dir)
#         tf.summary.trace_export(
#                 "train", step=batch, profiler_outdir=train_log_dir
#             )
#         tf.summary.scalar('gen_loss', gen_loss_m.result(), step=i)
#         tf.summary.scalar('perc_loss', perc_loss_m.result(), step=i)
#         tf.summary.scalar('disc_loss', disc_loss_m.result(), step=i)
#         train_summary_writer.flush()
                                                        
lm = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)


tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = train_log_dir,
      write_graph=True, # visualize the graph
     histogram_freq = 1, update_freq=100,
     profile_batch = (1,600))

# train(train_ds, EPOCHS)
model = GANModel(generator, discriminator)
model.compile(metrics=["gen_loss", "disc_loss", "ms_ssim", "ms"])
model.fit(train_ds.repeat(), epochs=400, callbacks=[tboard_callback, lm], steps_per_epoch=300)


In [ ]:
def change_model(model, new_input_shape=(None, 40, 40, 3)):
    # replace input shape of first layer
    # model.layers[1].batch_input_shape = new_input_shape
    # input_layer = layers.InputLayer(input_shape=new_input_shape, name="input_1")
    # model.input = input_layer

    new_model = make_sgenerator_model(new_input_shape)

    # feel free to modify additional parameters of other layers, for example...
    # model._layers[2].pool_size = (8, 8)
    # model._layers[2].strides = (8, 8)

    # rebuild model architecture by exporting and importing via json
    # new_model = keras.models.model_from_json(model.to_json())
    new_model.summary()

    # copy weights from old model to new one
    for layer in new_model.layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
        except:
            print("Could not transfer weights for layer {}".format(layer.name))

    # test new model on a random input image

    return new_model

In [ ]:
checkpoint_path = checkpoint_dir+"/weights/weights-{epoch:04d}.ckpt"

generator.save_weights(checkpoint_path.format(epoch=0))
discriminator.save_weights(checkpoint_path.format(epoch=0))

In [ ]:
save_path = "./weights/gan/2/{}/"

gen_path = os.path.join(save_path.format("generator"))
tf.saved_model.save(generator, gen_path)
disc_path = os.path.join(save_path.format("discriminator"))
tf.saved_model.save(discriminator, disc_path)


In [ ]:
loaded_generator = tf.saved_model.load(gen_path)
loaded_discriminator = tf.saved_model.load(disc_path)

In [ ]:
generate_and_save_images(loaded_generator, loaded_discriminator,
                             9999)